In [2]:
import pandas as pd # working with tabular data
import pickle # for machine learning models
import seaborn as sns # visualization
import matplotlib.pyplot as plt # visualization

from sklearn.feature_extraction import DictVectorizer # Machine Learning
from sklearn.linear_model import LinearRegression # Machine Learning
from sklearn.linear_model import Lasso # Regularization
from sklearn.linear_model import Ridge # Regularization

from sklearn.metrics import mean_squared_error # Loss Function

In [3]:
import mlflow

# to hook up with MLFlow UI
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("mlops_nyc_taxi") # choose a name for your experiment

2025/03/20 22:12:40 INFO mlflow.tracking.fluent: Experiment with name 'mlops_nyc_taxi' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///d:/KMUTT/3rd year/MLops_Class/cpe393-mlflow/mlruns/2', creation_time=1742483560023, experiment_id='2', last_update_time=1742483560023, lifecycle_stage='active', name='mlops_nyc_taxi', tags={}>

In [5]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [7]:
df_train = read_dataframe('./data/yellow_tripdata_2022-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2022-02.parquet')

In [8]:
len(df_train), len(df_val)

(2421440, 2918187)

In [9]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [29]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [30]:
import joblib
joblib.dump(dv, "dictvectorizer.pkl")

['dictvectorizer.pkl']

In [11]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [12]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred)

30.58376968375958

In [13]:
with open('./lin_reg.bin', 'wb') as f_out: # wb means write binary mlops-zoomcamp/week_1/models
    try:
        # Pickle both the dictionary vectorizer and the linear regression model
        pickle.dump((dv, lr), f_out)
        print("Model successfully pickled.")
    except Exception as e:
        print("Error occurred while pickling the model:", str(e))

Model successfully pickled.


In [14]:
!pip install xgboost
!pip install hyperopt

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   --- ------------------------------------ 13.6/150.0 MB 71.5 MB/s eta 0:00:02
   -------- ------------------------------- 31.2/150.0 MB 76.2 MB/s eta 0:00:02
   ------------- -------------------------- 51.6/150.0 MB 82.2 MB/s eta 0:00:02
   ------------------- -------------------- 73.1/150.0 MB 88.0 MB/s eta 0:00:01
   ---------------------- ----------------- 85.7/150.0 MB 81.6 MB/s eta 0:00:01
   -------------------------- ------------ 103.8/150.0 MB 81.9 MB/s eta 0:00:01
   ------------------------------ -------- 119.0/150.0 MB 80.8 MB/s eta 0:00:01
   ---------------------------------- ---- 133.7/150.0 MB 79.1 MB/s eta 0:00:01
   --------------------------------------  146.8/150.0 MB 76.9 MB/s eta 0:00:01
   --------------------------------------  149.9/150.0 MB 76.6 MB/s eta 0:00:01
   --------------------------------------  149.9/150.0 MB 76.6 MB/s eta 0:00:01
   --------------------------------------- 150.0/

In [15]:
# import required modules
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials # some methods to optimize hyperparameters
from hyperopt.pyll import scope

In [16]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [19]:
# Objective function - set the parameters for this specific run
def objective(params): 
    
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params) 
        booster = xgb.train(
            params=params,
            dtrain=train, # model trained on training set
            num_boost_round=3, # restricted due to time constraints - a value of 1000 iterations is common
            evals=[(valid, 'validation')], # model evaluated on validation set
            early_stopping_rounds=3 # if no improvements after 3 iterations, stop running # restricted, time constraints
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [25]:
# Set the range of the hyperparameter optimization search
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 200, 1)), # tree depth 4 to 100. Returns float, so convert to integer
    'learning_rate': hp.loguniform('learning_rate', -3, 0), # range exp(-3), exp(0) which is (0.049787, 1.0)
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1), # range exp(-5), exp(-1) which is (0.006738, 0.367879)
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1), # range exp(-6), exp(-1) which is (0.002479, 0.367879)
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3), # range exp(-1), exp(3) which is (0.367879, 20.085537)
    'objective': 'reg:linear',
    'seed': 69
}


best_result = fmin( # imported above
    fn=objective,
    space=search_space, # as defined above
    algo=tpe.suggest, # tpe is the algorithm used for optimization
    max_evals=5, #restricted, time constraints
    trials=Trials()
)

  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

d:\KMUTT\3rd year\MLops_Class\cpe393-mlflow\env\lib\site-packages\xgboost\callback.py:386: UserWarning: [23:07:01] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:7.37686                          
[1]	validation-rmse:6.16732                          
[2]	validation-rmse:5.57207                          
 20%|██        | 1/5 [03:41<14:45, 221.26s/trial, best loss: 31.047942096340115]

d:\KMUTT\3rd year\MLops_Class\cpe393-mlflow\env\lib\site-packages\xgboost\callback.py:386: UserWarning: [23:10:42] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:8.23158                                                     
[1]	validation-rmse:7.20195                                                     
[2]	validation-rmse:6.49829                                                     
 40%|████      | 2/5 [04:32<06:03, 121.23s/trial, best loss: 31.047942096340115]

...

[0]	validation-rmse:7.64447                                                     
[1]	validation-rmse:6.45597                                                     
[2]	validation-rmse:5.79610                                                     
 60%|██████    | 3/5 [10:10<07:20, 220.12s/trial, best loss: 31.047942096340115]

d:\KMUTT\3rd year\MLops_Class\cpe393-mlflow\env\lib\site-packages\xgboost\callback.py:386: UserWarning: [23:17:11] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:7.16800                                                     
[1]	validation-rmse:5.98813                                                     
[2]	validation-rmse:5.46996                                                     
 80%|████████  | 4/5 [11:29<02:44, 164.55s/trial, best loss: 29.92046618738642] 

d:\KMUTT\3rd year\MLops_Class\cpe393-mlflow\env\lib\site-packages\xgboost\callback.py:386: UserWarning: [23:18:30] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:8.62432                                                    
[1]	validation-rmse:7.78387                                                    
[2]	validation-rmse:7.13888                                                    
100%|██████████| 5/5 [11:40<00:00, 140.04s/trial, best loss: 29.92046618738642]


In [27]:
best_result

{'learning_rate': np.float64(0.42144329660840024),
 'max_depth': np.float64(78.0),
 'min_child_weight': np.float64(2.124315553751024),
 'reg_alpha': np.float64(0.2536733890391954),
 'reg_lambda': np.float64(0.20223267082622062)}

In [26]:
mlflow.xgboost.autolog(disable=True) # MLflow will not store parameters automatically - these will have to be requested

In [22]:
#best params

params = {
    'learning_rate': 0.33500504398153885,
    'max_depth': 74,
    'min_child_weight': 3.4593376740383635,
    'objective': 'reg:linear',
    'reg_alpha': 0.02439429006088758,
    'reg_lambda': 0.0029162481576915956,
    'seed': 42   
}

In [32]:
import numpy as np
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    # best_params = {
    #     'learning_rate': 0.4434065752589766,
    #     'max_depth': 81,
    #     'min_child_weight': 10.423237853746643,
    #     'objective': 'reg:linear',
    #     'reg_alpha': 0.2630756846813668,
    #     'reg_lambda': 0.1220536223877784,
    #     'seed': 42    
    # }

    best_params = {
        'learning_rate': 0.42144329660840024,
        'max_depth': 78,
        'min_child_weight': 2.124315553751024,
        'reg_alpha': 0.2536733890391954,
        'reg_lambda':0.20223267082622062,
        'seed': 69
 }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train, # model trained on training set
        num_boost_round=3, # restricted due to time constraints - a value of 1000 iterations is common
        evals=[(valid, 'validation')], # model evaluated on validation set
        early_stopping_rounds=3 # if no improvements after 3 iterations, stop running # restricted, time constraints
        )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("preprocessor.b", "wb") as f_out: # save pre-processing as a model
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("preprocessor.b", artifact_path="preprocessor") # we can isolate the pre-processing from raw data
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow") 

[0]	validation-rmse:7.16800
[1]	validation-rmse:5.98813
[2]	validation-rmse:5.46996


d:\KMUTT\3rd year\MLops_Class\cpe393-mlflow\env\lib\site-packages\mlflow\xgboost\__init__.py:168: UserWarning: [23:51:12] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/03/20 23:51:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [24]:
logged_model = 'mlruns/1/b0cbc283e72241b99cf27d0352dc1088/artifacts/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

OSError: No such file or directory: 'mlruns\1\b0cbc283e72241b99cf27d0352dc1088\artifacts\models_mlflow'

In [30]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: b0cbc283e72241b99cf27d0352dc1088

In [31]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

In [32]:
xgboost_model

In [33]:
y_pred = xgboost_model.predict(valid)

In [34]:
# check the first 10
y_pred[:10]

array([14.190828, 18.68855 , 26.193169, 23.354773, 26.193169, 13.043591,
       21.33266 ,  6.970524, 15.222571, 15.575311], dtype=float32)

Model Registry